<a href="https://colab.research.google.com/github/suwatoh/Python-learning/blob/main/127_%E4%B8%A6%E5%88%97%E3%82%BF%E3%82%B9%E3%82%AF%E5%AE%9F%E8%A1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

並列タスク実行
==============

multiprocessing
---------------

標準ライブラリの `multiprocessing` は、`threading` モジュールに似た API を使用してプロセスの生成をサポートするパッケージである。マルチプロセスでは GIL の影響を受けないので並列処理が行われ、CPU 資源を最大限に活用できる。

### プロセス開始方式

現在のプロセスから新たにプロセスを開始するとき、現在のプロセスを**親プロセス**、新たなプロセスを**子プロセス**という。プロセスが新しいタスクを引き受けた時、子プロセスを作成して処理させることができる。

`multiprocessing` はプロセスを開始するために以下の方法をサポートしている。

  * **fork**: `fork()` というシステムコールを呼び出すことで現在のプロセスをコピーする。プロセスの開始が速いが、プロセスのメモリ空間をそのままコピーするのでメモリ消費量が多い。Unix 系 OS でのみ利用可能。
  * **spawn**: 新たに Python インタプリタが動くプロセスを開始する。現在のモジュールは再読み込みされて変数も新たに作り直される。プロセスの開始に時間がかかるが、プログラムが動くのに必要なリソースのみ継承されるのでメモリ消費量を抑えられる。多くのプラットフォームで利用可能。

デフォルトの開始方式は、Unix 系 では fork、Windows では spawn。将来的には spawn に統一される予定。現在の開始方式名は `multiprocessing.get_start_method()` 関数で確認できる。開始方式を `method` に設定するには、`multiprocessing.set_start_method(method)` を実行する。これは一度しか呼び出すことができず、その場所もメインモジュールの `if __name__ == '__main__'` 節内で保護された状態でなければならない。

In [ ]:
import multiprocessing
multiprocessing.get_start_method()

'fork'

### インスタンス化

プロセスを使って子プロセスを作成するには、`multiprocessing.Process` クラスをインスタンス化する。

``` python
multiprocessing.Process(group=None, target=None, name=None, args=(), kwargs={}, *, daemon=None)
```

`multiprocessing.Process` クラスは、`threading.Thread` クラスと同様に使うことができる。ただし、以下のメソッドが加わっている:

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `terminate()` | プロセスを強制終了する。finally 句などは実行されないことに注意 | `None` |
| `close()` | `Process` オブジェクトを閉じ、関連付けられていたすべてのリソースを開放する。中のプロセスが実行中であった場合、`ValueError` を<br /><br />送出する | `None` |

次は、`threading.Thread` のインスタンス化のコード例を `multiprocessing.Process` クラスに書き換えただけである。

In [ ]:
import multiprocessing
import time


def run():
    a = []
    for i in range(20000000):
        a.append(0)


def main(*args):
    print(f"started at {time.strftime('%X')}")
    for t in args:
        t.start()
    for t in args:
        t.join()
    print(f"finished at {time.strftime('%X')}")


if __name__ == "__main__":
    t1 = multiprocessing.Process(target=run)
    t2 = multiprocessing.Process(target=run)
    t3 = multiprocessing.Process(target=run)
    print("2プロセス-----------")
    main(t1, t2)
    print("1プロセス-----------")
    main(t3)

2プロセス-----------
started at 02:40:04
finished at 02:40:08
1プロセス-----------
started at 02:40:08
finished at 02:40:10


このコードでは、各プロセスが動かす `run()` 関数は CPU バウンドな処理を行う。手元のマシンでこのコードを実行すると、2 個の子プロセスを同時に開始する場合と 1 個の子プロセスを開始する場合で処理にかかる時間がほとんど同じであった。これにより、CPU バウンドな処理でも並列処理されていること、つまり GIL の影響を受けないことがわかる。無料版 Colab では利用できる CPU コア数（論理コア数）が 2 個であるため、Python インタプリタを起動したプロセス（**メインプロセス**という）と子プロセス 1 個しか並列処理できず、2 個の子プロセスは並行処理されることに注意する。

``` python
multiprocessing.freeze_support()
```

この関数は、`multiprocessing` を使用しているプログラムをフリーズして Windows の実行可能形式を生成するためのサポートを追加する。サードパーティ製の PyInstaller などで実行可能形式を生成する場合に必要。この関数は、メインモジュールの `if __name__ == '__main__'` の直後に呼び出す必要がある。以下に例を示す:

``` python
from multiprocessing import Process, freeze_support

def f():
    print('hello world!')

if __name__ == '__main__':
    freeze_support()
    Process(target=f).start()
```

`freeze_support()` の呼び出しは、Unix 系 OS では効果がない。また、Windows の通常の Python インタプリタによって実行されているならば、`freeze_support()` は効果がない。

``` python
multiprocessing.current_process()
```

この関数は、現在のプロセスに対応する `Process` オブジェクトを返す。

``` python
multiprocessing.active_children()
```

この関数は、現在のすべてのアクティブな子プロセスのリストを返す。

``` python
multiprocessing.parent_process()
```

この関数は、現在のプロセスの親プロセスに対応する `Process` オブジェクトを返す。現在のプロセスがメインプロセスの場合、`None` を返す。

In [ ]:
from multiprocessing import Process, current_process
import time

def counter(data):
    data['n'] += 1
    print(f"{current_process().name}: {data['n']=}")
    time.sleep(0.2)

def main():
    data = {'n': 0}
    p = Process(target=counter, name='ChildProcess', args=(data,))
    p.start()
    p.join()
    print(f"{current_process().name}: {data['n']=}")

if __name__ == "__main__":
    main()

ChildProcess: data['n']=1
MainProcess: data['n']=0


このコードでは、子プロセスにおいて辞書 `data` の要素 `data['n']` をインクリメントしているが、その結果がメインプロセスに共有されず、メインプロセスでは `data['n']` は初期値 `0` のままである。fork では子プロセスに `data` のコピーが渡される（spawn では `data` が新たに作り直される）からである。

このように、マルチプロセスでは各プロセスが独立したメモリ空間を持つため、他のプロセスの影響を受けずに動作する。これにより、子プロセスの 1 つがクラッシュしても他の子プロセスやメインプロセスは影響を受けず、プログラム全体がダウンするリスクを減らすことができる。

### プロセスの同期

一般的にマルチプロセスプログラムでは、メモリが共有されないため競合状態が起こらず、マルチスレッドプログラムほどには同期プリミティブを必要としないが、`multiprocessing` は `threading` モジュールと等価な同期プリミティブを備えている。

  * `multiprocessing.Lock`
  * `multiprocessing.RLock`
  * `multiprocessing.Semaphore`
  * `multiprocessing.BoundedSemaphore`
  * `multiprocessing.Event`
  * `multiprocessing.Condition`
  * `multiprocessing.Barrier`

本来互いに独立しているプロセス間でこうした同期プリミティブが使えるのは、OS の機能を介して特定の一時ファイルにアクセスすることで実現されている。このようにプロセス間でデータのやり取りをする仕組みは**プロセス間通信**（Inter-Process Communication）、略して IPC と呼ばれる。IPC は、共有メモリを利用するだけのスレッド間通信と比べると重い処理となる。

次の例では、ロックを使用して、一度に 1 つのプロセスしか標準出力に書き込まないようにしている:

In [ ]:
from multiprocessing import Process, Lock, current_process
import time

def worker(lock, i):
    with lock:
        print(f"{current_process().name}: start")
        time.sleep(0.1)
        print(f"{current_process().name}: end")

if __name__ == "__main__":
    lock = Lock()
    for num in range(5):
        Process(target=worker, name=f"p{num}", args=(lock, num)).start()

p0: start
p0: end
p1: start
p1: end
p2: start
p2: end
p3: start
p3: end


ロックを使用しないで標準出力に書き込んだ場合は、各プロセスからの出力がごちゃまぜになってしまう。ただし、ロックの範囲ではマルチプロセスの性能を発揮できなくなる。

### パイプとキュー

**パイプ**（pipe）は、OS が提供する IPC の方式の 1 つであり、2 つのプロセスの入出力をつなぐ。`multiprocessing` は、パイプを利用するための関数 `Pipe` を提供している。

``` python
multiprocessing.Pipe(duplex=True)
```

パイプの両端を表すコネクションオブジェクトのペア `(conn1, conn2)` を返す。`conn1` と `conn2` は「接続」した状態となる。`duplex` が `True`（デフォルト）の場合、パイプは双方向性となる。`duplex` が `False` の場合、パイプは一方向性で、`conn1` はデータの受信専用、`conn2` はデータの送信専用になる。

コネクションオブジェクトは、以下のメソッドを持つ。

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `send(obj)` | 接続している相手にオブジェクトを送る。オブジェクトは `pickle` でシリアライズ可能でなければならな<br /><br />い。極端に大きすぎるオブジェクトでは `ValueError` 例外が送出されることがある。受信専用で `send()` <br /><br />を呼び出すと `OSError` 例外が発生する | `None` |
| `recv()` | 接続している相手側から送られたオブジェクトを返す。何か受け取るまで待機する。何も受け取らずに接<br /><br />続が相手側で閉じられた場合 `EOFError` 例外が発生する。送信専用で `recv()` を呼び出すと `OSError` <br /><br />例外が発生する | Unknown |
| `send_bytes(buf, offset=0, size=None)` | 接続している相手にバイトデータとして `buf` を送る。`offset` が指定されると `buf` のその位置から<br /><br />データが読み込まれる。`size` が指定されると `buf` からその量のデータが読み込まれる。極端に大き<br /><br />すぎるバイトデータでは `ValueError` 例外が送出されることがある。受信専用で `send_bytes()` を呼<br /><br />び出すと `OSError` 例外が発生する | `None` |
| `recv_bytes([maxlength])` | 接続している相手側から送られたデータをバイト列として返す。何も受け取らずに接続が相手側で閉じら<br /><br />れた場合 `EOFError` 例外が発生する。送信専用で `recv_bytes()` を呼び出すと `OSError` 例外が発生<br /><br />する。`maxlength` を指定していて、かつデータが `maxlength` より長い場合、`OSError` 例外が発生する | `bytes` |
| `close()` | 接続を閉じる | `None` |

OS による本来のパイプは一方向性であり、双方向性は Python 側の拡張である。2 つのプロセスが同時に同じパイプにデータを入れたり受け取ったりすると、データが破損する可能性がある。このような危険を回避したい場合は、`Pipe(duplex=False)` として通信方向を制限するとよい。

`recv()` の処理ではデシリアル化が行われる。それはデータを送ったプロセスが信頼できる場合を除いてセキュリティリスクになることに注意する。これが問題となるなら、バイト列限定となるが `send_bytes()` と `recv_bytes()` を使うとよい。

`recv()` や `recv_bytes()` でデータを受け取る順番は、データを送った順番と同じ、つまり先入れ先出し（FIFO）となる。

In [ ]:
from multiprocessing import Process, Pipe

def worker(conn):
    conn.send([42, None, 'hello'])
    conn.send_bytes(b'thank you')
    conn.close()

def  main():
    parent_conn, child_conn = Pipe()
    p = Process(target=worker, args=(child_conn,))
    p.start()
    assert parent_conn.recv() == [42, None, 'hello']
    assert parent_conn.recv_bytes() == b'thank you'
    p.join()

if __name__ == "__main__":
    main()

また、 `multiprocessing` は、パイプや 2～3 個のロック/セマフォを使用して実装されたプロセス共有キュー `Queue` も提供している。この `Queue` クラスは `queue.Queue` と同様に使用できるが、`task_done()` と `join()` メソッドがないことに注意。`put()` での例外に `queue.Full`、`get()` での例外に `queue.Empty` が使用されるが、それらは `multiprocessing` の名前空間では利用できないため、これらの例外を捕捉する場合は `queue` からインポートする必要がある。

In [ ]:
from multiprocessing import Process, Queue
import time

def writer(q):
    data = []
    data.append([42, None, 'hello'])
    data.append(b'thank you')
    for msg in data:
        q.put(msg)
        time.sleep(0.5)

def reader(q):
    while True:
        print(q.get())

def main():
    q = Queue()
    pw = Process(target=writer, args=(q,))
    pr = Process(target=reader, args=(q,))
    pw.start()
    pr.start()
    # pwが完了するのを待つ（q.join()はないことに注意）
    pw.join()
    # prを終了（readerが無限ループなので）
    pr.terminate()

if __name__ == "__main__":
    main()

[42, None, 'hello']
b'thank you'


### 共有オブジェクト

実は、Unix 系 OS と Windows では、メモリ上の特定の領域を複数のプロセスからアクセス可能とする共有メモリをサポートしている。共有メモリを利用すると、データのシリアル化/デシリアル化を必要とするパイプに比べて、パフォーマンスが大幅に向上する。ただし、以下の点に注意する。

  * 共有メモリを使用すると、競合状態がマルチプロセスでも発生することになる。しかも、マルチプロセスでは並列処理が行われるので、共有メモリに対する操作が不可分であること（**アトミック**ともいう）に注意がより必要である。ここにアトミックとは、「操作に途中の状態がなく、一部のみが失敗するということがない」ことである。たとえば、代入演算 `=` はアトミックであるが、`+=` のような演算は読み込みと書き込みを含むためアトミックではない。共有データに対するアトミックでない操作がロックを伴わないと、他のプロセスによる操作が割り込んで、意図しない結果になる可能性がある。
  * 共有メモリは OS の機能なので、C のデータ型やデータ構造が利用される。

`multiprocessing` は、共有メモリを使用するオブジェクト（**共有オブジェクト**と呼ぶ）をサポートする。次の 2 つの関数は、それぞれ共有メモリに割り当てられた数値と数値の配列を扱う共有オブジェクトを返す。

``` python
multiprocessing.Value(typecode_or_type, *args, lock=True)
multiprocessing.Array(typecode_or_type, size_or_initializer, *, lock=True)
```

`typecode_or_type` に数値の C データ型を指定する。`ctypes` モジュールで使用できる型か、`array` モジュールで使用されるような 1 文字の型コードを指定できる。

キーワード専用引数 `lock` が `True`（デフォルト）なら、値へ同期アクセスするために新たに `RLock` オブジェクトが作成される。

`Value()` 関数が返すオブジェクトは、 `value` 属性で値を参照できる。`Array()` 関数が返すオブジェクトは、 `[]` によるインデックス参照で値を参照でき、スライスを使うこともできる。どちらのオブジェクトも、`get_lock()` メソッドで内部ロックオブジェクトにアクセスできる。

`Value` と `Array` の使用例:

In [ ]:
from multiprocessing import Process, Value, Array


def worker(n, a):
    # アトミックではない操作はロックを必要とする
    with n.get_lock():
        n.value += 1

        for i in range(len(a)):
            a[i] = -a[i]


def main():
    num = Value("i", 0)
    arr = Array("d", [0.1, 0.2, 0.3])

    p = Process(target=worker, args=(num, arr))
    p.start()
    p.join()

    print(f"{num.value = }")
    print(f"{arr[:] = }")


if __name__ == "__main__":
    main()

num.value = 1
arr[:] = [-0.1, -0.2, -0.3]


### マネージャー

**マネージャー**（manager）は、共有オブジェクトの高レベルな使い方をサポートする。マネージャーでは、共有オブジェクトを管理する子プロセスが使用される。このプロセスがサーバープロセスとなって、他のプロセスはサーバープロセスを通して共有オブジェクトにアクセスすることになる。これにより高レベルな使い方ができる反面、サーバープロセスを使うことによるオーバーヘッドが発生すること、IPC にパイプが使われるためデータのシリアル化/デシリアル化によるオーバーヘッドも発生することに注意する。

マネージャークラスは、`multiprocessing.managers.BaseManager` クラスの派生クラスとして定義される。

以下は、`BaseManager` クラスで定義されている主なメソッド。

``` python
BaseManager.start(initializer=None, initargs=())
```

サーバープロセスを開始する。`initializer` が `None` でなければ、サーバープロセスは開始時に `initializer(*initargs)` を呼び出す。

``` python
BaseManager.shutdown()
```

サーバープロセスを停止する。

``` python
BaseManager.get_server()
```

マネージャーの制御下にある実際のサーバーを表す `Server` オブジェクトを返す。`Server` オブジェクトは `serve_forever()` メソッドをサポートする。

``` python
BaseManager.connect()
```

ローカルからリモートのマネージャーオブジェクトへ接続する。

``` python
BaseManager.register(typeid, callable=None, proxytype=None, exposed=None, method_to_typeid=None, create_method=True)
```

クラスメソッド。「プロキシを返すメソッド」をマネージャークラスに登録するために使用される。**プロキシ**は、共有オブジェクトを参照するオブジェクトであり、自身は組み込み型オブジェクトなどの Python オブジェクトと同様に操作される。プロキシを使うと Python らしい書き方で共有オブジェクトを操作することができる。

既に多くの「プロキシを返すメソッド」が登録済みであるマネージャークラス `multiprocessing.managers.SyncManager` が用意されている。これを使えば、プログラマーが `register()` を使ってマネージャークラスを定義する必要がない。ただし、直接インスタンス化するのではなく、次の関数の戻り値としてインスタンスを得ること。

``` python
multiprocessing.Manager()
```

「プロキシを返すメソッド」を使用する前に、`start()` メソッドでサーバープロセスを開始しておく必要がある。プロキシを使用する必要がなくなったときは、`shutdown()` でサーバープロセスを停止する。全てのマネージャーは、コンテキストマネージャーとして使用できる。`__enter__()` は `start()` メソッドを呼び出してからマネージャーオブジェクトを返す。また `__exit__()` は `shutdown()` を呼び出す。

次のコード片

``` python
with Manager() as manager:
    # do something...
```

これは、以下と同じ。

``` python
manager = Manager()
manager.start()
try:
    # do something...
finally:
    manager.shutdown()
```

`SyncManager` は、 `dict` や `list`、`multiprocessing.managers.Namespace` などに対応するプロキシを返すメソッドが登録されている。メソッドの名前はそれぞれの型の名前と同じになっている。 `multiprocessing.managers.Namespace` は、ドット演算子 `.` を使用する属性参照で共有オブジェクトを参照するために使用される。

`Manager()` の使用例:

In [ ]:
from multiprocessing import Process, Manager


def f(dct, lst, ns):
    dct[1] = "1"
    dct["2"] = 2
    dct[0.25] = None
    lst.reverse()
    ns.x = 10
    ns.y = "hello"


def main():
    with Manager() as manager:
        dct = manager.dict()
        lst = manager.list(range(10))
        ns = manager.Namespace()

        p = Process(target=f, args=(dct, lst, ns))
        p.start()
        p.join()

        print(f"{dct=!s}")
        print(f"{lst=!s}")
        print(f"{ns.x=}, {ns.y=}")


if __name__ == "__main__":
    main()

dct={1: '1', '2': 2, 0.25: None}
lst=[9, 8, 7, 6, 5, 4, 3, 2, 1, 0]
ns.x=10, ns.y='hello'


`BaseManager` を継承して独自のマネージャーを作成し、それをネットワーク経由で他のコンピューター上のプロセスによって共有することもできる。以下のコード例では、ローカルマシンで実行可能とするため、アドレスを `'localhost'` としている。

server.py（共有キューのためにサーバーを作成）:

``` python
from multiprocessing.managers import BaseManager
from queue import Queue

queue = Queue()

class QueueManager(BaseManager):
  pass

# キューに対応するプロキシを返すメソッド get_queue を登録
QueueManager.register('get_queue', callable=lambda: queue)

# マネージャーを取得
manager = QueueManager(address=('localhost', 50000), authkey=b'abracadabra')
# サーバーを取得
server = manager.get_server()
# サーバーを起動する
server.serve_forever()
```

client1.py（データをキューに登録）:

``` python
from multiprocessing.managers import BaseManager

class QueueManager(BaseManager):
    pass

# メソッド get_queue を登録
QueueManager.register('get_queue')

# マネージャーを取得
manager = QueueManager(address=('localhost', 50000), authkey=b'abracadabra')
# サーバーへ接続
manager.connect()
queue = manager.get_queue()
queue.put('hello')
```

client2.py（データをキューから取り出す）:

``` python
from multiprocessing.managers import BaseManager

class QueueManager(BaseManager):
    pass

# メソッド get_queue を登録
QueueManager.register('get_queue')

# マネージャーを取得
manager = QueueManager(address=('localhost', 50000), authkey=b'abracadabra')
# サーバーへ接続
manager.connect()
queue = manager.get_queue()
print(queue.get())
```

ターミナルを 3 つ起動し、まず 1 つのターミナルで `python server.py` を実行してサーバーを立ち上げ、残りのターミナルでそれぞれ `python client1.py` と `python client2.py` を実行する。`client2.py` のプロセスはキューに値がない場合は待ち状態になり、`client1.py` のプロセスで `'hello'` が put されたら、それを出力する。

### 共有メモリ管理

共有オブジェクトを介して共有メモリを利用できるのであるが、直接に共有メモリそのものを管理することもできる。共有メモリを直接管理することによって、マネージャーを利用しなくても柔軟に共有メモリを利用することができる。

`multiprocessing.shared_memory` モジュールは、共有メモリそのものを管理するための `SharedMemory` クラスを提供する。

``` python
multiprocessing.shared_memory.SharedMemory(name=None, create=False, size=0)
```

| 引数 | 意味 |
|:---|:---|
| `name` | 共有メモリの一意の名前を文字列で指定する。`None`（デフォルト）の場合、新しい名前が構成される |
| `create` | `True` の場合、新しい共有メモリの領域を確保して、それに結び付いたインスタンスを作成する。`False`（デフォルト）の場合、既存の共有メモリに結び<br /><br />付いたインスタンスを作成する |
| `size` | 新しい共有メモリの領域を確保するときに要求されるバイト数。実際に確保されるサイズはこれより大きくなることがある。既存の共有メモリを使用<br /><br />する場合は、`size` は無視される |

コンストラクタの `name` と `create` 引数を使うことで、あるプロセスが特定の名前で共有メモリを作成し、別のプロセスが同じ名前を使用して同じ共有メモリを参照することができる。

| 属性 | 意味 |
|:---|:---|
| `buf` | 共有メモリの内容 |
| `name` | 読み取り専用。共有メモリの名前（文字列） |
| `size` | 読み取り専用。実際に割り当てられた共有メモリのサイズ |

`buf` のデータ構造は C の配列であり、オブジェクトの参照ではなくオブジェクトの値そのものが格納される。インデックス参照が可能で、スライスも使える。たとえば、`buf[:4]` は先頭から 4 バイトのデータを参照する。

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `close()` | このインスタンスから共有メモリへのアクセスを閉じる。`close()` を呼び出しても共有メモリ領域自体は破棄されない | `None` |
| `unlink()` | OS に共有メモリの破棄を要求する。このメソッドは全てのインスタンスのうちいずれかが 1 回だけ呼び出すこと。`unlink()` と `close()`<br /><br /> はどちらの順序でも呼び出すことができる | `None` |

共有メモリがどのプロセスでも必要なくなった場合は、適切なクリーンアップが実行されるために `unlink()` メソッドを呼び出す必要がある。OS は、共有メモリがどのプロセスからもアクセスされる可能性がないことを確認してから共有メモリを破棄する。このため、全てのインスタンスは共有メモリが不要になったら `close()` を呼び出す必要がある。

`SharedMemory` の使用例:

In [ ]:
import array
from multiprocessing import Process, current_process
from multiprocessing.shared_memory import SharedMemory


def worker(name: str):
    # 共有メモリを構成（既存の共有メモリを使用）
    shm = SharedMemory(name=name)
    print("{}: {}, {}, {}, {}".format(current_process().name, *shm.buf[:4]))
    shm.buf[:6] = b"Python"

    #  共有メモリが不要になったら閉じる
    shm.close()


def main():
    # 共有メモリを構成
    shm = SharedMemory(create=True, size=10)
    print(f"Size of SharedMemor: {shm.size}")
    shm.buf[:3] = array.array("B", [11, 22, 33])
    shm.buf[3] = 44
    assert (shm.buf[0], shm.buf[1], shm.buf[2], shm.buf[3]) == (11, 22, 33, 44)

    p = Process(target=worker, name="ChildProcess", args=(shm.name,))
    p.start()
    p.join()
    assert (shm.buf[0], shm.buf[1], shm.buf[2], shm.buf[3]) == (80, 121, 116, 104)
    print("{}: {}".format(current_process().name, bytes(shm.buf[:6])))

    #  共有メモリが不要になったら閉じる
    shm.close()

    # 共有メモリを破棄
    shm.unlink()


if __name__ == "__main__":
    main()

Size of SharedMemor: 10
ChildProcess: 11, 22, 33, 44
MainProcess: b'Python'


バイト単位で共有メモリを構成する場合、データサイズが 2 バイト以上のデータ型（例: `float`）を扱うことが難しい。そこで、`multiprocessing.shared_memory` モジュールは、Python の組み込みデータ型からなる固定長リストの形で共有メモリを構成するための `ShareableList` クラスを提供する。`ShareableList` は `SharedMemory` のラッパークラスになっている。

``` python
multiprocessing.shared_memory.ShareableList(sequence=None, *, name=None)
```

`sequence` で与えた順番で値が格納された共有メモリ（`name` で一意の名前を付けられる）に結び付けられたオブジェクトを作成する。格納可能な値は次の組み込みデータ型に制限される。

  * `int` （ただし符号付き 64 ビット整数）
  * `float`
  * `bool`
  * `str` （ただし UTF-8 エンコードしたとき 10 MB 未満のもの）
  * `bytes` （ただし 10 MB 未満のもの）
  * `None`

既存の `ShareableList` に結び付ける場合は、`sequence` を `None` に設定したまま、`name` で共有メモリの一意の名前を指定する。

| 属性 | 意味 |
|:---|:---|
| `shm` | 内部で作成された `SharedMemory` インスタンス。`close()` や `unlink()` を呼び出すために使われる |

`ShareableList` オブジェクトは変更可能なリストのように使える（イテラブルであり、`[]` を使ったインデックス参照も可能）。しかし、全体の長さを変更することはできない（つまり、`append()`、`insert()` などを使用できない）。また、スライスによる新しい `ShareableList` インスタンスの動的な作成をサポートしていない。

`str` や `bytes` の値を参照する場合、末尾の連続した `\x00`（ヌル文字またはヌルバイト）は削除される。`str` や `bytes` の値を変更する場合、もとの値より短い値に変更するのであれば末尾が `\x00` で埋められるが、もとの値より長い値に変更しようとすれば `ValueError` が発生する。あらかじめ必要な文字数（バイト数）を確保するために末尾に連続した `\x00` を入れるとよい。ただし、現在、`ShareableList` のバグにより、`sequence` の末尾以外の位置に `\x00` で埋めた `str` や `bytes` の値を指定した場合に、その値をインデックス参照すると、`\x00` と一緒に後ろに格納された値も削除されてしまう。このため、`\x00` で埋めた `str` や `bytes` の値は `sequence` の末尾に置くとよい。

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `count(value)` | `value` の出現回数を返す | `int` |
| `index(value)` | `value` の最初のインデックス位置を返す。値が存在しない場合は `ValueError` を発生させる | `int` |

`ShareableList` の使用例:

In [ ]:
from multiprocessing import Process, current_process
from multiprocessing.shared_memory import ShareableList


def worker(name: str):
    # 共有メモリを構成（既存の共有メモリを使用）
    a = ShareableList(name=name)
    print("{}: {}, {}, {}, {}, {}".format(current_process().name, a[0], a[1], a[2], a[3], a[4]))

    # データを変更
    a[0] = 0
    a[1] = 3.141592
    a[2] = False
    a[3] = b"aaa"
    a[4] = "Hello World"

    #  共有メモリが不要になったら閉じる
    a.shm.close()


def main():
    # 共有メモリを構成
    a = ShareableList([100, -273.154, True, b"aaabbb", "Hello\x00\x00\x00\x00\x00\x00"])
    assert a.count(100) == 1
    assert a.index(True) == 2

    p = Process(target=worker, name="ChildProcess", args=(a.shm.name,))
    p.start()
    p.join()
    print("{}: {}, {}, {}, {}, {}".format(current_process().name, a[0], a[1], a[2], a[3], a[4]))

    #  共有メモリが不要になったら閉じる
    a.shm.close()

    # 共有メモリを破棄
    a.shm.unlink()


if __name__ == "__main__":
    main()

ChildProcess: 100, -273.154, True, b'aaabbb', Hello
MainProcess: 0, 3.141592, False, b'aaa', Hello World


`unlink()` 忘れを回避するために、マネージャーを使うことができる。`multiprocessing.managers.SharedMemoryManager` はマネージャーで、`start()` メソッドでサーバープロセスを開始してから以下のメソッドで `SharedMemory` や `ShareableList` のインスタンスを作成することができ、また、`shutdown()` メソッドで `unlink()` を呼び出す。

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `SharedMemory(size)` | `size` バイトの大きさを要求する新しい `SharedMemory` インスタンスを作成して返す | `SharedMemory` |
| `ShareableList(sequence)` | `sequence` の値で初期化された新しい `ShareableList` インスタンスを作成して返す | `ShareableList` |

`SharedMemoryManager` をコンテキストマネージャーとして with 文で使用する場合、`start()` と `shutdown()` が暗黙的に実行される。

次のコードは、`ShareableList` の使用例を `SharedMemoryManager` を利用する形で書き換える例である。

In [ ]:
from multiprocessing import Process, current_process
from multiprocessing.managers import SharedMemoryManager


def worker(sl):
    print("{}: {}, {}, {}, {}, {}".format(current_process().name, sl[0], sl[1], sl[2], sl[3], sl[4]))

    # データを変更
    sl[0] = 0
    sl[1] = 3.141592
    sl[2] = False
    sl[3] = b"aaa"
    sl[4] = "Hello World"


def main():
    # 共有メモリ管理プロセスを開始
    with SharedMemoryManager() as smm:
        sl = smm.ShareableList([100, -273.154, True, b"aaabbb", "Hello\x00\x00\x00\x00\x00\x00"])
        assert sl.count(100) == 1
        assert sl.index(True) == 2

        p = Process(target=worker, name="ChildProcess", args=(sl,))
        p.start()
        p.join()
        print("{}: {}, {}, {}, {}, {}".format(current_process().name, sl[0], sl[1], sl[2], sl[3], sl[4]))


if __name__ == "__main__":
    main()

ChildProcess: 100, -273.154, True, b'aaabbb', Hello
MainProcess: 0, 3.141592, False, b'aaa', Hello World


### プロセスプール

子プロセスの作成は重い処理となるため、少ない子プロセスを使い回す仕組み、いわれる**プロセスプール**を利用すべきである。`multiprocessing` パッケージはプロセスプールを作成するためのクラス `Pool` を提供する。

``` python
multiprocessing.Pool(processes=None, initializer=None, initargs=(), maxtasksperchild=None)
```

| 引数 | 意味 |
|:---|:---|
| `processes` | プールされる子プロセスの数。省略した場合、`os.cpu_count()` によって返される数が使用される |
| `initializer`,<br /><br /> `initargs` | `initializer` が `None` ではない場合、各子プロセスは開始時に `initializer(*initargs)` を呼び出す |
| `maxtasksperchild` | タスクが完了したら終了してもよい子プロセスの数。省略した場合、プロセスがプールと同じ期間だけ生き続ける |

以下は、`Pool` オブジェクトのメソッド。

``` python
Pool.apply(func, args=(), kwds={})
```
プール内の 1 つの子プロセスを使って、引数 `args` とキーワード引数 `kwds` を伴って `func` を呼ぶ。`func` の戻り値を返す。このメソッドは、終了するまで後続の処理をブロックする。

``` python
Pool.apply_async(func, args=(), kwds={}, callback=None, error_callback=None)
```

`apply()` の非同期版で、`multiprocessing.pool.ApplyResult` クラスのインスタンス（結果オブジェクト）を返す。`callback` と `error_callback` には 1 個の引数を受け取る呼び出し可能オブジェクトを指定できる。結果を返せるようになったときに `callback` が結果オブジェクトに対して適用される。ただし呼び出しが失敗した場合は、例外インスタンスを伴って `error_callback` が適用される。

``` python
Pool.map(func, iterable, chunksize=None)
```

組み込み関数 `map()` の並列版。ただし、`iterable` 引数は 1 つだけサポートされる（`func` の引数は 1 つだけサポートされるということである）。`iterable` から `chunksize `の長さ分だけタスクが切り出されて各プロセスに割り振られる。デフォルトでは、プールされるプロセス数の 4 倍で切り出される。

``` python
Pool.map_async(func, iterable, chunksize=None, callback=None, error_callback=None`)
```

`map()` メソッドの非同期版。`multiprocessing.pool.MapResult` クラスのインスタンス（結果オブジェクト）を返す。`callback` と `error_callback` については `apply_async()` と同様。

``` python
Pool.imap(func, iterable, chunksize=1)
```

`map()` の遅延評価版。`func` の戻り値を yield するイテレーターを返す。`chunksize` のデフォルト値は 1 とされる。

``` python
Pool.imap_unordered(func, iterable, chunksize=1)
```

イテレーターが返す結果の順番が任意の順番でよいと見なされることを除けば `imap()` と同じ。

``` python
Pool.starmap(self, func, iterable, chunksize=None)
```

`iterable` の要素が引数としてアンパックされるイテレート可能オブジェクトであると期待される以外は `map()` メソッドと同様。そのため、`iterable` が `[(1,2), (3, 4)]` なら、結果は `[func(1,2), func(3,4)]` になる。

``` python
Pool.starmap_async(self, func, iterable, chunksize=None, callback=None, error_callback=None)
```

`starmap()` メソッドの非同期版。引数と戻り値は `map_async()` と同様。

``` python
Pool.close()
```

これ以上プールでタスクが実行されないようにする。すべてのタスクが完了した後で子プロセスが終了する。

``` python
Pool.terminate()
```

実行中の処理を完了させずに子プロセスをすぐに停止する。

``` python
Pool.join()
```

子プロセスが終了するのを待つ。`join()` を使用する前に `close()` か `terminate()` を呼び出す必要がある。

なお、`Pool` オブジェクトはコンテキストマネージャーとして使用できる。`__enter__()` は `Pool` オブジェクトを返す。また `__exit__()` は `terminate()` を呼び出す。

次のコードは、`apply()` メソッドの使用例である:

In [ ]:
from multiprocessing import Pool, current_process
from random import random
import time

def worker(x):
    print("{}(x={}) working...".format(current_process().name, x))
    time.sleep(random())
    return x * x

def main():
    # 同時に最大2個の子プロセス
    with Pool(processes=2) as pool:
        results = []
        for i in range(10):
            res = pool.apply(func=worker, args=(i,))
            results.append(res)
        print(results)
    pool.join()

if __name__ == "__main__":
    main()

ForkPoolWorker-20(x=0) working...
ForkPoolWorker-21(x=1) working...
ForkPoolWorker-20(x=2) working...
ForkPoolWorker-21(x=3) working...
ForkPoolWorker-20(x=4) working...
ForkPoolWorker-21(x=5) working...
ForkPoolWorker-20(x=6) working...
ForkPoolWorker-21(x=7) working...
ForkPoolWorker-20(x=8) working...
ForkPoolWorker-21(x=9) working...
[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]


`apply()` で指定した `worker()` 関数は `random()` 関数で処理時間が異なるようにしている。それにもかかわらず、出力結果から、2 つの子プロセスが交互に結果を返していること、つまり `apply()` が同期処理を行うことがわかる。マルチプロセスの並列処理には `apply_async()` のほうが適している。`apply()` は、処理の順序付けをしたい場合に使う。

`apply_async()` で返される `ApplyResult` オブジェクトは、以下のメソッドを持つ。

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `get([timeout])` | 結果を返すが、結果を受け取るまで待ち、結果を受け取ったときに返す。引数に `timeout` を指定すると、結果が `timeout` 秒以<br /><br />内に受け取れない場合に `multiprocessing.TimeoutError` が発生する | 呼び出し<br /><br />の結果 |
| `wait([timeout])` | その結果が有効になるか timeout 秒経つまで待つ | `None` |
| `ready()` | その呼び出しが完了しているかどうかを返す | `bool` |
| `successful()` | その呼び出しが例外を発生させることなく完了したかどうかを返す。その結果が返せる状態でない場合 `ValueError` が発生する | `bool` |

次のコードは、`apply_async()` メソッドの使用例である:

In [ ]:
from multiprocessing import Pool, current_process
from random import random
import time

def worker(x):
    print("{}(x={}) working...".format(current_process().name, x))
    time.sleep(random())
    return x * x

def main():
    # 同時に最大2個の子プロセス
    with Pool(processes=2) as pool:
        results = []
        for i in range(10):
            res = pool.apply_async(func=worker, args=(i,))
            results.append(res)
        # 結果の取得
        reply = []
        for res in results:
            try:
                reply.append(res.get(timeout=2.0))
            except TimeoutError:
                print("タイムアウトしました")
    pool.join()
    print(reply)

if __name__ == "__main__":
    main()

ForkPoolWorker-22(x=0) working...ForkPoolWorker-23(x=1) working...

ForkPoolWorker-22(x=2) working...
ForkPoolWorker-23(x=3) working...
ForkPoolWorker-23(x=4) working...
ForkPoolWorker-22(x=5) working...
ForkPoolWorker-22(x=6) working...
ForkPoolWorker-22(x=7) working...
ForkPoolWorker-23(x=8) working...
ForkPoolWorker-22(x=9) working...
[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]


`apply_async()` で指定したタスクは、先にタスクを完了した子プロセスに次々に割り振られていく様子がわかる。また、`ApplyResult` オブジェクトの `get()` メソッドが結果を受け取るまで待つので、順番通りに結果を取得できることもわかる。

`apply()` メソッドの使用例と `apply_async()` メソッドの使用例では、子プロセスで動かす関数が 1 引数なので、for 文で繰り返している部分を、それぞれ `map()` メソッドと `map_async()` メソッドで書き換えることができる。`map_async()` で返される `MapResult` オブジェクトは、 `ApplyResult` のサブクラスで、 `get()` メソッドが結果のリストを返すようにオーバーライドされる。

次のコードは、`map_async()` メソッドの使用例である:

In [ ]:
from multiprocessing import Pool, current_process
from random import random
import time

def worker(x):
    print("{}(x={}) working...".format(current_process().name, x))
    time.sleep(random())
    return x * x

def main():
    # 同時に最大2個の子プロセス
    with Pool(processes=2) as pool:
        results = pool.map_async(func=worker, iterable=range(10))
        # 結果の取得
        try:
            reply = results.get(timeout=5.0)
        except TimeoutError:
            print("タイムアウトしました")
    pool.join()
    print(reply)

if __name__ == "__main__":
    main()

ForkPoolWorker-24(x=0) working...
ForkPoolWorker-25(x=2) working...
ForkPoolWorker-24(x=1) working...
ForkPoolWorker-25(x=3) working...
ForkPoolWorker-24(x=4) working...
ForkPoolWorker-24(x=5) working...
ForkPoolWorker-25(x=6) working...
ForkPoolWorker-25(x=7) working...
ForkPoolWorker-24(x=8) working...
ForkPoolWorker-24(x=9) working...
[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]


### multiprocessing.dummy

`multiprocessing.dummy` サブパッケージを利用すると、 `multiprocessing` の API を利用してマルチスレッドを実現できる。

実は、`multiprocessing.Pool` のサブクラスとして、スレッドプールを使用する `multiprocessing.pool.ThreadPool` クラスが定義されており、`multiprocessing.dummy.Pool()` は、この `ThreadPool` のインスタンスを返す。つまり、以下の 2 つのコードは等価である。

``` python
from multiprocessing.dummy import Pool
pool = Pool(processes, initializer, initargs)
```

``` python
from multiprocessing.pool import ThreadPool
pool = ThreadPool(processes, initializer, initargs)
```

`queue.Queue` を利用して作成したスレッドプールと同じような動作を `Pool()` で実現できる。

In [ ]:
from multiprocessing.dummy import Pool
import threading
import time


def worker(item):
    print(f"{threading.current_thread().name}: Working on {item}")
    time.sleep(item * 0.1)
    print(f"{threading.current_thread().name}: Finished {item}")


def main():
    # スレッドプールを設定（2 つのスレッドからなる）
    with Pool(processes=2) as pool:
        results = pool.map_async(func=worker, iterable=range(1, 6))
        # スレッドの開始
        results.get()
    # 終了を待機
    pool.join()
    print("全てのタスクが完了した")


if __name__ == "__main__":
    main()

Thread-29 (worker): Working on 1
Thread-30 (worker): Working on 2
Thread-29 (worker): Finished 1
Thread-29 (worker): Working on 3
Thread-30 (worker): Finished 2
Thread-30 (worker): Working on 4
Thread-29 (worker): Finished 3
Thread-29 (worker): Working on 5
Thread-30 (worker): Finished 4
Thread-29 (worker): Finished 5
全てのタスクが完了した


フューチャーパターン
--------------------

`Pool` の `apply_async()` や `map_async()` で作成される結果オブジェクト（`ApplyResult` インスタンスや `MapResult` インスタンス）は、作成時にはタスクの結果を保持していなくても未来にはその結果を受け取ることができるという不思議なオブジェクトである。このようなオブジェクトは、しばしば future と呼ばれ、以下のようなデザインパターンの下でそのクラスが設計される。

**フューチャーパターン**（future pattern）は、プロキシパターンの一種で、並列処理のためのデザインパターンである。値を取得できる既存クラスに対して、そのラッパーであるクラス（Future クラスと呼ぶ）を使って、次のような機能を追加する。

  1. 既存クラスのインスタンスから値を取得できる場合は、その値を保持する。
  2. 既存クラスのインスタンスから値を取得できない場合は、値を取得できる状態になるまで待つ。

このように、Future クラスは待機状態を持つ。待機状態により値の取得を後回しにすることができ、これにより並列処理でのやり取りをスムーズに行うことができる。

たとえば、ある計算を行うクラス X があって、その計算を要求する処理（receiver）と実際の計算を行う処理（sender）の間で以下のようなやり取りをする。

  * （receiver） X クラスをラップする Future クラスをインスタンス化し、それを何らかの手段を用いて sender に渡す。
  * （sender） X オブジェクトの計算を行い、Future オブジェクトの待機状態を解除する。
  * （receiver） Future オブジェクトを確認し、待機状態が解除されていれば Future オブジェクトから計算結果を取得できるが、待機状態である間はブロックされる。

X で行う計算が時間のかかるものである場合、Future オブジェクトを使うことで並列処理が効率的に行われる。

Future の機能はよく引換券に例えられる（future は現物に対する先物という意味もある）。食券を発行する食堂で考えると、料理を要求する客と、料理を提供する食堂スタッフがいて、食堂ではすぐには料理が渡されないので、客は券売機で食券を購入して待つ。配膳口に食券の番号が表示されたら、客は配膳口で料理を受け取ることができる。食券システムによって客と食堂スタッフのやり取りがスムーズになり、客は料理ができるまでのスキマ時間を有効に活用することもできる。

多くのプログラミング言語では、フューチャーパターンの実装が言語の機能あるいはライブラリとして取り込まれている。

concurrent.futures
------------------

標準ライブラリの `concurrent.futures` モジュールは、`multiprocessing.Pool` や `multiprocessing.dummy.Pool` の機能限定版となるインターフェースを提供する。スレッド間通信やプロセス間通信を使って細かい制御を行う必要がない、あるいは、そのような制御を使わないという制限を課す場合には、`concurrent.futures` モジュールが適している。

### Future

`concurrent.futures` モジュールが提供するインターフェースを使用すると、`concurrent.futures.Future` オブジェクトが作成される。これは、フューチャーパターンの Python 実装であり、タスクの未来の値を表す。`multiprocessing.pool.ApplyResult` より高機能である。モジュールの利用者がこのオブジェクトを直接作成する必要はなく、また操作するメソッドは次のメソッドに限られる。

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `cancel()` | 呼び出しのキャンセルを試みる。呼び出しが現在実行中または実行が終了していてキャンセルできない場合、この<br /><br />メソッドは `False` を返し、そうでない場合、呼び出しはキャンセルされ、このメソッドは `True` を返す | `bool` |
| `cancelled()` | 呼び出しが正常にキャンセルされた場合 `True` を返す | `bool` |
| `running()` | 現在呼び出しが実行中でキャンセルできない場合 `True` を返す | `bool` |
| `done()` | 呼び出しが正常にキャンセルされたか終了した場合 `True` を返す | `bool` |
| `result(timeout=None)` | 呼び出しによって返された値を返す。もし呼び出しがまだ完了していなければ、このメソッドは待機する。`timeout`<br /><br /> に浮動小数点数を指定した場合、タイムアウトすると `TimeoutError` が送出される。`Future` オブジェクトが完了す<br /><br />る前にキャンセルされた場合、`CancelledError` が送出される。呼び出しが例外を送出した場合、このメソッドは同<br /><br />じ例外を送出する | Unknown |
| `exception(timeout=None)` | 呼び出しによって送出された例外を返す。もし呼び出しがまだ完了されていなければ、このメソッドは待機する。<br /><br />`timeout` に浮動小数点数を指定した場合および `Future` オブジェクトが完了する前にキャンセルされた場合は<br /><br /> `result()` と同様。呼び出しが例外を送出することなく完了した場合、`None` を返す | `Exception`<br /><br /> &#124; `None` |
| `add_done_callback(fn)` | 完了時コールバックとして呼び出し可能なオブジェクト `fn` を追加する。`Future` オブジェクトがキャンセルされた<br /><br />か、完了した際に、`Future` オブジェクトをそのただ 1 つの引数として `fn` が呼び出される。追加された完了時コー<br /><br />ルバックは、追加された順番で、追加を行ったプロセスに属するスレッド中で呼び出される。もし `Future` オブジェ<br /><br />クトが既に完了しているか、キャンセル済みであれば、`fn` は即座に実行される | `None` |

### Executor

`concurrent.futures.Executor` は、スレッドまたはプロセスのプールを使用して非同期に呼び出しを行うインターフェースを提供するクラスである。以下のメソッドを持つ。

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `submit(fn, /, *args, **kwargs)` | 呼び出し可能オブジェクト `fn` を `fn(*args, **kwargs)` として実行するようにスケジュ<br /><br />ールし、`concurrent.futures.Future` オブジェクトを返す | `Future` |
| `map(fn, *iterables, timeout=None, chunksize=1)` | 組み込み関数 `map()` の非同期版。`fn` の実行結果（`Future` オブジェクトの `result()`<br /><br /> メソッドの戻り値）を返すジェネレーターを返す。返す順番は `iterables` で渡すタスクの<br /><br />順番となる（実行が完了した順番ではない）。`fn` 呼び出しで例外が発生した場合、その値<br /><br />がジェネレーターから取得される時にその例外が発生する。`chunksize` はプロセスプー<br /><br />ルでのみ有効で `Pool.map()` と同じ | ｼﾞｪﾈﾚｰﾀｰ |
| `shutdown(wait=True, *, cancel_futures=False)` | シャットダウンする。以後 `submit()` と `map()` を呼び出すと `RuntimeError` が発生する。<br /><br />`cancel_futures` が `False`（デフォルト）の場合、実行を開始せず保留中の `Future` オブ<br /><br />ジェクトがキャンセルされず、それらの全てが完了してリソースが解放される。`wait` が<br /><br /> `True`（デフォルト）の場合、リソース解放までメソッドは返らない | `None` |

`Executor` は、コンテキストマネージャーとして使用できる。`shutdown()` は with ブロックを終了するときに呼び出される。

`Executor` クラスを直接使ってはならず、サブクラスである `ThreadPoolExecutor`（マルチスレッド）か `ProcessPoolExecutor` （マルチプロセス）を介して使うこと。

``` python
concurrent.futures.ThreadPoolExecutor(max_workers=None, thread_name_prefix='', initializer=None, initargs=())
```

| 引数 | 意味 |
|:---|:---|
| `max_workers` | プールされるスレッドの数を指定する。`None` か指定を省略する場合のデフォルト値 `min(32, os.cpu_count() + 4)` |
| `thread_name_prefix` | スレッド名の接頭辞を指定する |
| `initializer` | スレッド開始時に実行する関数を指定する |
| `initargs` | `initializer` の引数をタプルで指定する |

``` python
concurrent.futures.ProcessPoolExecutor(max_workers=None, mp_context=None, initializer=None, initargs=(), max_tasks_per_child=None)
```

| 引数 | 意味 |
|:---|:---|
| `max_workers` | プールされるプロセスの数を指定する。`None` か指定を省略する場合のデフォルト値はマシン上の CPU コア数になる。<br /><br />Windows では 61 以下に制約される |
| `mp_context` | プロセス生成時の開始方式を指定する |
| `initializer` | プロセス開始時に実行する関数を指定する |
| `initargs` | `initializer` の引数をタプルで指定する |
| `max_tasks_per_child` | Python 3.11 で追加。1 つのプロセスが実行できるタスクの最大数を指定する。この数を超えるとプロセスは終了する。<br /><br />`None`（デフォルト）の場合、プロセスはプールと同じ期間存続する |

`ProcessPoolExecutor` については、次の 3 点に注意する。

  * `ProcessPoolExecutor` は、Python インタプリターの対話モードでは動作しない。
  * `ProcessPoolExecutor` は、`multiprocessing.Queue` を利用しているため、pickle 化できるオブジェクトを利用する必要がある。pickle 化できないファイルオブジェクトやラムダ式などは、`ProcessPoolExecutor` で実行する呼び出し可能オブジェクト、その引数と戻り値として利用できない。
  * `ProcessPoolExecutor` に渡された呼び出し可能オブジェクトから `Executor` や `Future` のメソッドを呼ぶとデッドロックに陥る。

次のコードは、あえて効率の悪い素数判定アルゴリズムを用いた `is_prime()` 関数を、比較のため、シングルプロセスとマルチプロセスで動かす。マルチプロセスでは、さらに `submit()` メソッドと `map()` メソッドで動かしている。

In [ ]:
from concurrent.futures import ProcessPoolExecutor
import time

PRIMES = [
    10897409,
    20292113,
    31518271,
    40003027,
]

def is_prime(n: int) -> bool:
    """効率の悪い素数判定"""
    if n < 2 or not isinstance(n, int):
        raise ValueError("nは2以上の整数を指定してください")
    i = 2
    while i < n:
        if n % i == 0:
            return False
        i += 1
    return True

def main():
    # シングルプロセスで実行
    print(f"single process started at {time.strftime('%X')}")
    for n in PRIMES:
        if is_prime(n):
            print(f"{n} is prime")
    print(f"single process finished at {time.strftime('%X')}")

    # マルチプロセス（submit メソッド）で実行
    print(f"multi process (submit) started at {time.strftime('%X')}")
    with ProcessPoolExecutor() as executor:
        n_futures = []
        for n in PRIMES:
            n_futures.append((n, executor.submit(is_prime, n)))
        for n, future in n_futures:
            if future.result():
                print(f"{n} is prime")
    print(f"multi process (submit) finished at {time.strftime('%X')}")

    # マルチプロセス（map メソッド）で実行
    print(f"multi process (map) started at {time.strftime('%X')}")
    with ProcessPoolExecutor() as executor:
        for n, result in zip(PRIMES, executor.map(is_prime, PRIMES)):
            if result:
                print(f"{n} is prime")
    print(f"multi process (map) finished at {time.strftime('%X')}")

if __name__ == "__main__":
    main()

single process started at 02:40:22
10897409 is prime
20292113 is prime
31518271 is prime
40003027 is prime
single process finished at 02:40:38
multi process (submit) started at 02:40:38
10897409 is prime
20292113 is prime
31518271 is prime
40003027 is prime
multi process (submit) finished at 02:40:51
multi process (map) started at 02:40:51
10897409 is prime
20292113 is prime
31518271 is prime
40003027 is prime
multi process (map) finished at 02:41:05


`submit()` メソッドは、1 個のタスクしか実行できないので、複数のタスクを同時に実行するには for 文を使うなどの工夫が必要である。

`map()` メソッドは、イテラブルが返すタスクを実行できる。`multiprocessing.Pool` の `map_async()` に似ているが、複数の iterable 引数をサポートする（実行する `fn` 関数が複数の引数をとることができる）。`map()` メソッドはジェネレーターを返すが、そのジェネレーターは `Future` オブジェクトを返すのではなく、`Future` オブジェクトの `result()` メソッドの戻り値、つまり呼び出しの結果を返すことに注意する。上記のコードでは、結果表示のために、与えたタスクとその結果の組が欲しかったので、組み込み関数 `zip()` を使っている。`zip()` 関数が問題なく使えるのは、`map()` が返すジェネレーターでは完了した順番ではなく渡したタスクの順番で結果を受け取るからである。

Colab の制約により、上の実行結果にはマルチプロセスの効果が表れなかった。手元のマシンでは、上記のコードを実行した結果にマルチプロセスの効果が表れたことを確認している。

上記のコードは、`ProcessPoolExecutor` を `ThreadPoolExecutor` に単純に置換するだけで、マルチスレッドで動作する。ただし、`is_prime()` 関数は CPU バウンドな処理を行うので、GIL によってマルチスレッドの効果は表れない。

さて、上記のコードの `is_prime()` 関数をタプル `tuple[int, bool]` を返すように変更する。この場合、`submit()` メソッドでは、完了時コールバックを追加し、完了時コールバックの中で結果を取得して上記コードと同様の結果表示を行える。また、`map()` メソッドでは、`zip()` を使う必要はない。

In [ ]:
from concurrent.futures import Future, ProcessPoolExecutor
import time

PRIMES = [
    10897409,
    20292113,
    31518271,
    40003027,
]

def is_prime(n: int) -> tuple[int, bool]:
    """効率の悪い素数判定"""
    if n < 2 or not isinstance(n, int):
        raise ValueError("nは2以上の整数を指定してください")
    i = 2
    while i < n:
        if n % i == 0:
            return n, False
        i += 1
    return n, True

def my_callback_function(future: Future):
    n, result = future.result()
    if result:
        print(f"{n} is prime")

def main():
    # マルチプロセス（submit メソッド）で実行
    print(f"multi process (submit) started at {time.strftime('%X')}")
    with ProcessPoolExecutor() as executor:
        for n in PRIMES:
            future = executor.submit(is_prime, n)
            future.add_done_callback(my_callback_function)
    print(f"multi process (submit) finished at {time.strftime('%X')}")

    # マルチプロセス（map メソッド）で実行
    print(f"multi process (map) started at {time.strftime('%X')}")
    with ProcessPoolExecutor() as executor:
        for n, result in executor.map(is_prime, PRIMES):
            if result:
                print(f"{n} is prime")
    print(f"multi process (map) finished at {time.strftime('%X')}")

if __name__ == "__main__":
    main()

multi process (submit) started at 14:07:12
10897409 is prime
20292113 is prime
31518271 is prime
40003027 is prime
multi process (submit) finished at 14:07:25
multi process (map) started at 14:07:25
10897409 is prime
20292113 is prime
31518271 is prime
40003027 is prime
multi process (map) finished at 14:07:38


### as_completed

複数のタスクを同時に実行し、完了した `Future` オブジェクトから先に効率よく結果を受け取るということは、単純な `submit()` メソッドの繰り返しや `map()` メソッドを使う限りは実現できない。この場合、次のモジュール関数を使うとよい。

``` python
concurrent.futures.as_completed(fs, timeout=None)
```

この関数は、`Future` オブジェクトのイテレーター `fs` を受け取り、新たに完了順に `Future` オブジェクトを返すジェネレーターを返す。このジェネレーターから受け取る `Future` オブジェクトでは `result()` メソッドの呼び出しから直ちに結果を得る（または例外が発生する）。なお、`fs` は、異なる `Executor` インスタンスによって作成された `Future` オブジェクトを返すものであってもよい。

`timeout` 引数を指定した場合、`timeout` 秒経過してもジェネレーターが `Future` オブジェクトを返さないとき、`concurrent.futures.TimeoutError` 例外を発生させる。

以下は、HTTP リクエストを伴う大量のタスクのバッチ処理を想定したコード例である。処理が I/O バウンドなので、`ThreadPoolExecutor` で並列処理を行っている。例外処理も行うようにしている。ここでは、 [httpstat.us](https://httpstat.us/) にアクセスし HTTP Status Code のレスポンスを得ているだけである。 httpstat.us は、URL に欲しいステータスコードの番号を付けてアクセスするだけで、そのステータスコードのレスポンスを返してくれるサービスを提供している。`https://httpstat.us/200` なら `200 OK` のステータスコードである。`sleep=<millisecond>` の形でクエリ文字列を追加すると、`<millisecond>` ミリ秒経過までレスポンスを遅らせることができる。

In [ ]:
from concurrent.futures import Future, ThreadPoolExecutor, TimeoutError, as_completed
import urllib.request

def my_task(millisecond: int) -> tuple[str, int]:
    if millisecond < 0:
        raise ValueError(f"millisecond is {millisecond}, must be greater than or equal to 0")
    with urllib.request.urlopen(f"https://httpstat.us/200?sleep={millisecond}") as response:
        body = response.read().decode("utf-8")
        return body, millisecond

def my_add_done_callback(future: Future):
    try:
        result = future.result()
    except ValueError as err:
        print(f"{type(err).__name__}: {err}")
    else:
        print(f"Task completed: {result}")

def main():
    # タスクのリスト（遅延する時間（ミリ秒）を指定する）
    tasks = [3000, 1000, 2000, 4000, 6000, -1000]
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(my_task, millisecond=task) for task in tasks]
        try:
            # タイムアウトを 6 秒に設定するので、おそらく 6000 のタスクはタイムアウトする
            for future in as_completed(futures, timeout=6.0):
                future.add_done_callback(my_add_done_callback)
        except TimeoutError as err:
            print(f"{type(err).__name__}: {err}")

if __name__ == "__main__":
    main()

ValueError: millisecond is -1000, must be greater than or equal to 0
Task completed: ('200 OK', 1000)
Task completed: ('200 OK', 2000)
Task completed: ('200 OK', 3000)
Task completed: ('200 OK', 4000)
TimeoutError: 1 (of 6) futures unfinished


呼び出しで発生した例外は、`Future` オブジェクトの `result()` メソッドで結果を取得する際に処理する。`as_completed()` 関数で発生した `concurrent.futures.TimeoutError` 例外は、関数の呼び出しを try-except 文で書いて処理する。

### 排他制御

マルチスレッドでは、競合状態が発生する場合、ロックを使用する必要がある。

In [ ]:
from concurrent.futures import ThreadPoolExecutor
from threading import Lock, current_thread
import time

def counter(data: dict, lock: Lock):
    with lock:  # ロックの獲得と解放
        n = data['n']
        time.sleep(0.2)
        data['n'] = n + 1
        print(f"{current_thread().name}: {data['n']=}")

def main():
    data = {"n": 0}
    lock = Lock()  # ロックを作成
    with ThreadPoolExecutor() as executor:
        for _ in range(3):
            executor.submit(counter, data, lock)
    print(f"{current_thread().name}: {data['n']=}")

if __name__ == "__main__":
    main()

ThreadPoolExecutor-1_0: data['n']=1
ThreadPoolExecutor-1_1: data['n']=2
ThreadPoolExecutor-1_2: data['n']=3
MainThread: data['n']=3


subprocess
----------

標準ライブラリの `subprocess` モジュールは、新しいプロセスの開始、入力・出力・エラーパイプの接続、リターンコードの取得を可能とする。

### run

``` python
subprocess.run(args, *, **kwargs)
```

このモジュール関数は、`args`（文字列のリストまたは単一の文字列）で指定されたコマンドを実行する。コマンドの完了を待って、`subprocess.CompletedProcess` インスタンスを返す。

`args` 以外の引数は、すべてキーワード専用引数となっており、主なものは次のとおり（全てのキーワード専用引数は[公式ドキュメント](https://docs.python.org/ja/3/library/subprocess.html#using-the-subprocess-module)を参照）。

| kwargs | 意味 | default |
|:---|:---|:--:|
| `stdin` | 標準入力を指定する。以下から選ぶ<br /><br />・`None`: リダイレクトは発生しない<br /><br />・`subprocess.PIPE`: 子プロセスへの新しいパイプを作成する<br /><br />・`subprocess.DEVNULL`: `os.devnull` を使用する<br /><br />・ファイル記述子（正の整数）<br /><br />・有効なファイル記述子を持つファイルオブジェクト | `None` |
| `stdout` | 標準出力を指定する。`stdin` と同じ項目を選べる | `None` |
| `stderr` | 標準エラー出力を指定する。`stdout` で選べる項目に加え、`subprocess.STDOUT` も選べる。これは子プロセスからの `stderr` の<br /><br />内容が `stdout` と同じファイルハンドルにキャプチャされる必要があることを示す | `None` |
| `capture_output` | `True` の場合、標準出力と標準エラー出力の内容がそれぞれ `CompletedProcess` インスタンスの `stdout` 属性と `stderr` 属性で<br /><br />参照できる | `False` |
| `shell` | `args` が単一の文字列で `shell` が `True` の場合、その文字列がシェルによって実行される。`args` が単一の文字列で `shell` が<br /><br /> `False`（デフォルト）の場合、その文字列は引数を指定せずに実行される単なるプログラムの名前でなければならない | `False` |
| `cwd` | 作業ディレクトリのパスを表す path-like オブジェクトを指定する | `None` |
| `timeout` | `timeout` 秒経過後に処理が完了しなかった場合に `subprocess.TimeoutExpired` 例外を送出する。`timeout` が `None`（デフォル<br /><br />ト）の場合、タイムアウトしない | `None` |
| `check` | `True` の場合、子プロセスが非ゼロの終了コードで終了したなら、`subprocess.CalledProcessError` 例外を送出する | `False` |

`subprocess.CompletedProcess` の属性:

| 属性 | 意味 |
|:---|:---|
| `args` | プロセスを起動するときに使用された引数。1 個のリストか 1 個の文字列になる |
| `returncode` | 子プロセスの終了コード。正常終了の場合 `0` を返す |
| `stdout` | 子プロセスから補足された標準出力の内容（バイト列） |
| `stderr` | 子プロセスから補足された標準エラー出力の内容（バイト列） |

`subprocess.CompletedProcess` のメソッド:

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `check_returncode()` | 終了コードが非ゼロの場合、`subprocess.CalledProcessError` が送出される | `None` |

次は、`capture_output` 引数を設定して、標準出力と標準エラー出力を捕捉するコード例である。

In [ ]:
import subprocess
obj = subprocess.run(["ls", "-l"], capture_output=True)
print("stdout:\n{}".format(obj.stdout.decode("utf-8")))
print("stdout:\n{}".format(obj.stderr.decode("utf-8")))  # 捕捉した内容がない場合は None を返す

stdout:
total 4
drwxr-xr-x 1 root root 4096 Jul  1 13:21 sample_data

stdout:



これは、 `stdout` 引数と `stderr` 引数に `subprocess.PIPE` を設定する次のコードと同じである。

In [ ]:
import subprocess
obj = subprocess.run(["ls", "-l"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
print("stdout:\n{}".format(obj.stdout.decode("utf-8")))
print("stdout:\n{}".format(obj.stderr.decode("utf-8")))  # 捕捉した内容がない場合は None を返す

stdout:
total 4
drwxr-xr-x 1 root root 4096 Jul  1 13:21 sample_data

stdout:



次は、`check` 引数を設定して、子プロセスが正常終了しなかった場合に `subprocess.CalledProcessError` 例外を送出するコード例である。

In [ ]:
import subprocess
try:
    # シェルでコマンド "exit 1" を実行する
    obj = subprocess.run("exit 1", shell=True, check=True)
except subprocess.CalledProcessError as err:
    print(f"{type(err).__name__}: {err}")

CalledProcessError: Command 'exit 1' returned non-zero exit status 1.


これは、戻り値オブジェクトの `check_returncode()`メソッド を呼び出す次のコードと同じである。

In [ ]:
import subprocess
# シェルでコマンド "exit 1" を実行する
obj = subprocess.run("exit 1", shell=True)
try:
    obj.check_returncode()
except subprocess.CalledProcessError as err:
    print(f"{type(err).__name__}: {err}")

CalledProcessError: Command 'exit 1' returned non-zero exit status 1.


次は、`timeout` 引数を設定して、タイムアウトするコード例である。

In [ ]:
import subprocess
try:
    subprocess.run(["sleep", "10"], timeout=1.0)
except subprocess.TimeoutExpired as err:
    print(f"{type(err).__name__}: {err}")

TimeoutExpired: Command '['sleep', '10']' timed out after 0.9999655629999893 seconds


### Popen

実のところ、`subprocess.run()` 関数は、内部的にそのキーワード引数を伴い `subprocess.Popen` インスタンス化を呼び出している。`subprocess.run()` 関数よりも高度な処理を行いたい場合は、`subprocess.Popen` クラスを直接利用する。

`subprocess.Popen` は、`subprocess.CompletedProcess` と同様の属性を持ち、さらに `pid` 属性で子プロセスのプロセス ID を参照できる。

`subprocess.Popen` のメソッド:

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `poll()` | 子プロセスの実行が終了したら終了コードを返す。終了してないなら `None` を返す | `int` &#124; `None` |
| `wait(timeout=None)` | 子プロセスが終了するまで待機し、終了したら終了コードを返す。`timeout` を設定した場合、<br /><br />プロセスが `timeout` 秒経過後に終了してない場合、`TimeoutExpired` 例外を送出する | `int` &#124; `None` |
| `communicate(input=None, timeout=None)` | 子プロセスが終了するまで待機し、標準出力と標準エラー出力を読み込む | 標準出力と標準エラー<br /><br />出力の組となるタプル |
| `send_signal(signal)` | `signal` を子プロセスに送る | `None` |
| `terminate()` | 子プロセスを停止する | `None` |
| `kill()` | 子プロセスを強制終了する | `None` |

シェル上で `|` を使ったパイプライン処理は、2 つの子プロセスの標準出力と標準入力をパイプでつなぐことで記述することができる。

In [ ]:
import subprocess
# 2つの子プロセスをパイプで繋ぐ
p1 = subprocess.Popen(["ls", "-l", "sample_data"], stdout=subprocess.PIPE)
p2 = subprocess.Popen(["grep", "json"], stdin=p1.stdout, stdout=subprocess.PIPE)
out, err = p2.communicate()  # ls -l sample_data | grep json
print(out.decode())

-rwxr-xr-x 1 root root     1697 Jan  1  2000 anscombe.json



ファイルロック
--------------

### ファイルロックの意義

複数のプロセスが同時に同じファイルにアクセスすると、意図しない上書きや部分的なデータ損失が発生し、アプリケーションの誤動作を引き起こす。

例えば、現金の出納を記録する CSV 形式が次のようなものであったとする。

``` text
日付,摘要,入金額,出金額,残高
2025-04-01,前月繰越,150_000,,150_000
```

プロセス A およびプロセス B が同時に CSV 形式をファイルから読み込み、まずプロセス A が 50000 円の入金を計算してファイルにそのレコード全体を書き戻すと CSV 形式は次のようになる。

``` text
日付,摘要,入金額,出金額,残高
2025-04-01,前月繰越,150_000,,150_000
2025-04-07,預金より現金補充,50_000,,200_000
```

その後、プロセス B が 100000 円の出金を記録してファイルにそのレコード全体を書き戻すと、ファイルがプロセス B によって上書きされ、プロセス A が加えた変更が失われてしまう。

``` text
日付,摘要,入金額,出金額,残高
2025-04-01,前月繰越,150_000,,150_000
2025-04-07,事務所備品（PC）購入,,100_000,50_000
```

プロセス A とプロセス B が共に `multiprocessing` によって生成された子プロセスであれば、`multiprocessing.Lock` を使ってファイルの更新をシリアライズ（逐次化）することでこの問題を防ぐことができる。しかし、この解決法はプロセス A とプロセス B が互いに外部プロセスである場合には使えない。

そこで、ファイル自体についてプロセスによる更新をシリアライズする方法が必要となる。この方法が**ファイルロック**（file lock）である。

### ファイルロックの種類

ファイルロックは、複数のプロセスが同時に取得できるかどうかの違いにより、次のように種類分けされる。

  1. **共有ロック**（shared lock）:
      * 複数のプロセスが同時に共有ロックを取得できる。
      * このロックがかけられている間、他のプロセスは排他ロックを取得できない。
  2. **排他ロック** / **占有ロック** / **専有ロック**（exclusive lock）:
      * 同時に 1 つのプロセスのみが取得可能。
      * ロック解放まで、他のプロセスは共有ロック、排他ロックのいずれも取得できない。

次の表は、2 つのプロセスが同時にロックを取得できるかどうかを表してる。

| | 共有ロック | 排他ロック |
|:---|:--:|:--:|
| 共有ロック | ⭕ | ❌ |
| 排他ロック | ❌ | ❌ |

通常は、ファイル読み取り時には共有ロックが取得され、書き込みも行う時には排他ロックが取得される。

また、ファイルロックは、他のプロセスがすでにロックしている場合に待つかどうかの違いにより、次のように種類分けされる。

  * **ブロッキング**（blocking）：
  他のプロセスがすでにロックしている場合、解除されるまで待つ（タイムアウトするまでロックをリトライする）。
  * **ノンブロッキング**（non-blocking）：
  他のプロセスがすでにロックしている場合、直ちにエラーが発生する。

### 強制ロック

Windows は、強制的にファイルのアクセスを制御することが可能である。これを**強制ロック**（mandatory lock）という。

Windows API の関数 `CreateFile()` でファイルを開く際に引数 `dwShareMode` で共有モード（読み取り、書き込み、削除）を指定し、それ以外のアクセスを拒否できる。ファイルを閉じると、設定した共有モードによるアクセス制限が解除される。

また、ロック単位はファイル全体または部分（バイト単位）で指定することも可能である。

Python からファイルロックに関する Windows API を呼び出すには、標準ライブラリの `msvcrt` モジュールが提供する次の関数を使用する。

``` python
msvcrt.locking(fd, mode, nbytes)
```

| 引数 | 意味 |
|:---|:---|
| `fd` | ファイルディスクリプタ |
| `mode` | ロックの種類 |
| `nbytes` | ロックのバイト数。現在のファイル位置からのバイト数までとして指定する |

`mode` 引数には以下のモジュール定数を指定することができる。

| モジュール定数 | ロックの種類 | ブロッキング |
|:---|:---|:---|
| `msvcrt.LK_LOCK` | 排他ロック | ブロッキング |
| `msvcrt.LK_RLCK` | 共有ロック | ブロッキング |
| `msvcrt.LK_NBLCK` | 排他ロック | ノンブロッキング |
| `msvcrt.LK_NBRLCK` | 共有ロック | ノンブロッキング |
| `msvcrt.LK_UNLCK` | ロック解除 | |

ブロッキングの場合、Windows API では 1 秒ごとに 10 回までロックを試みる。ロックできなければ `OSError` が発生する。ノンブロッキングの場合、直ちに `OSError` が発生する。

ファイル全体にロックをかけたい場合は、先頭にシークしてサイズ分を指定する。

``` python
import msvcrt

with open("data.txt", "r+") as f:
    f.seek(0, 2)  # ファイル末尾へ（サイズ取得用）
    size = f.tell()
    f.seek(0)  # 先頭に戻す

    try:
        msvcrt.locking(f.fileno(), msvcrt.LK_LOCK, size)  # 全体ロック
        # 安全に読み書き
    finally:
        msvcrt.locking(f.fileno(), msvcrt.LK_UNLCK, size)  # ロック解除
```

強制ロックの場合、ロック中に他プロセスが読み書きしようとすると失敗するので、安全性が高い。その反面、OS が毎回ロック状態をチェックするため、オーバーヘッドがある。また、プロセスがロックを解除しないために起こる「ファイルの使用中」エラーに遭遇しやすい。

### アドバイザリーロック

Linux や macOS は、ファイルについてロック/アンロックという状態（ビットフラグ）を変更することができるが、プロセスに強制しない。これを**アドバイザリーロック**（advisory lock）という。ロックに従うかどうかはアプリケーションの設計次第であり、ロックを無視して読み書きすることも技術的には可能である。

Linux や macOS が強制ロックを採用しない理由は、POSIX 準拠のファイルパーミッションと強制ロックを両立させる実装が難しいからであると言われている。

ファイルの状態（ビットフラグ）の変更や確認は `fcntl()` システムコールを使う。

Python からファイルロックに関して C の `fcntl()` システムコールを行うには、標準ライブラリの `fcntl` モジュールが提供する次の関数を使用する。

``` python
fcntl.lockf(fd, cmd, len=0, start=0, whence=0)
```

| 引数 | 意味 |
|:---|:---|
| `fd` | ファイルディスクリプタ（または `fileno()` メソッドを提供しているファイルオブジェクト） |
| `cmd` | 操作の種類 |
| `len` | ロックのバイト数。`0`（デフォルト）の場合、ファイルの終了までロックすることを表す |
| `start` | ロック領域先頭の `whence` からの相対的なバイトオフセット。デフォルトは `0` |
| `whence` | `io.FileIO.seek()` の同名引数と同じ。デフォルトは `0`（ファイルの先頭位置） |

`cmd` 引数には以下のモジュール定数を指定することができる。

| 引数 | 意味 |
|:---|:---|
| `fcntl.LOCK_EX` | 排他ロック |
| `fcntl.LOCK_SH` | 共有ロック |
| `fcntl.LOCK_NB` | ノンブロッキング。`fcntl.LOCK_EX` か `fcntl.LOCK_SH` とビット演算子 `|` による論理和で指定 |
| `fcntl.LOCK_UN` | ロック解除 |

この関数は、ロックの取得に失敗すると `BlockingIOError` 例外を送出する。

In [ ]:
# 適当にファイルを作成しておく
!touch data.txt

import fcntl

with open("data.txt", "r+") as f:
    fcntl.lockf(f, fcntl.LOCK_EX)  # 排他ロック（ブロッキング）
    # 安全に読み書き
    fcntl.lockf(f, fcntl.LOCK_UN)  # ロック解除

### ロックファイル

OS によってファイルロックの実装、挙動が異なるため、クロスプラットフォーム対応なアプリケーションでは問題となる。

そこで、強制ロックにもアドバイザリーロックにも依存しないで独自にファイルロック機能を実装することがある。具体的には、「一時的なファイルを新規作成できたらロックの取得とみなす」という方法で実装する。この一時的なファイルを**ロックファイル**（lock file）と呼ぶ。

次のコードは、シンプルなロックファイル方式の実装例である。

In [ ]:
# 適当にファイルを作成しておく
!touch data.txt

import os
from pathlib import Path

class FileLock:
    def __init__(self, lock_file_path):
        self.lock_file = Path(lock_file_path)

    def acquire(self):
        """ロックを取得する"""
        try:
            # 'x' モードを指定してロックファイルを排他的に作成（既に存在する場合は失敗）
            with open(self.lock_file, 'x') as f:
                # プロセスIDを書き込んでおく（デバッグ用）
                f.write(str(os.getpid()))
            return True
        except FileExistsError:
            # 既にロックが存在する場合
            return False

    def release(self):
        """ロックを解放する"""
        try:
            self.lock_file.unlink()
            return True
        except FileNotFoundError:
            # ロックファイルが存在しない場合
            return False

if __name__ == "__main__":
    # 基本的な使用方法
    file = "data.txt"
    lockfile = "data.txt.lock"

    lock = FileLock(lockfile)

    if lock.acquire():
        try:
            with open("data.txt", "r+") as f:
                ... # 安全に読み書き
        finally:
            lock.release()

`FileLock` の `acquire()` メソッドでロックを取得する（成功時に `True`、失敗時に `False` を返す）。ロックファイルが既に存在する場合は `FileExistsError` が発生するため、これでロック状態を判定している。タイムアウト付きリトライ機能は含んでいない。

`FileLock` の `release()` メソッドでロックを解放する。

ロックファイル方式の使用方法において、ファイルの読み書きの部分を全く別のタスクに置き換えても問題がない。こうして、ロックファイル方式はファイルロック以外にも応用できる。例えば、プロセスの多重起動防止、リソースの排他利用、複数ジョブ間の依存関係制御（ジョブ A が完了したらロックファイルを削除、ジョブ B はその削除を待つ）など。

ロックファイル方式はプラットフォームに依存しない反面、プロセスの強制終了などでロックファイルが残るとデッドロックの原因になるため、注意が必要。

### filelock

サードパーティ製 [filelock](https://py-filelock.readthedocs.io/en/latest/) パッケージは、高機能で安全なロックファイル方式の実装を提供している。ライセンスは Unlicense。インストール方法は次のとおり。

``` python
pip install filelock
```

「ロックファイルの存在」だけでロック状態とみなす方式では、プロセスが強制終了したときにデッドロックが発生する可能性が高くなる。そこで、`filelock` は「ロックファイルの存在」ではなく「ロックファイルの排他アクセス」をロック状態とみなす。この方式ではロックを解除するときにロックファイルを削除する必要がない。

`filelock.FileLock` クラスは、クロスプラットフォーム対応のロックファイル方式を実装するもので、Windows 環境では `msvcrt` を使用する `filelock.WindowsFileLock` クラスの別名であり、それ以外の環境では `fcntl` を使用する `filelock.UnixFileLock` クラスの別名である。コンストラクタは次のとおり。

``` python
filelock.FileLock(lock_file, timeout=-1, mode=420, thread_local=True, *, blocking=True, is_singleton=False)
```

主な引数は次のとおり。

| 引数 | 意味 |
|:---|:---|
| `lock_file` | ロックファイルのパス（文字列またはファイルオブジェクト）。このファイルが存在しない場合は作成され、ロックに使用される |
| `timeout` | ロック獲得のために待機する最大秒数（`float`）。その間にロックを取得できなければ `filelock.Timeout` 例外を送出する。`0` の場合、ロックを<br /><br />取得できなければ待機せずに直ちに `filelock.Timeout` 例外を送出する。`-1`（デフォルト）の場合、無限に待つ |

主なメソッドは次のとおり。

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `acquire(timeout=None, poll_interval=0.05, *, poll_intervall=None, blocking=None)` | ロックを取得する | `AcquireReturnProxy` |
| `release()` | ロックを解放する。ロックファイルは<br /><br />自動的には削除されない | `None` |

`filelock.FileLock` オブジェクトはコンテキストマネージャーであるため with 構文をサポートする。`__enter__()` メソッドはオブジェクトの `acquire()` メソッドを呼び出した後にオブジェクト自身を返す。`__exit__()` メソッドはオブジェクトの `release()` メソッドを実行する。

次のコードは `filelock.FileLock` の使用例:

In [ ]:
# 適当にファイルを作成しておく
!touch data.txt

from filelock import FileLock

lock = FileLock("data.txt.lock", timeout=10)
with lock:
    with open("data.txt", "r+") as f:
        ... # 安全に読み書き